**Loading the dataset and custom transforms**

https://docs.lightly.ai/tutorials/package/tutorial_simsiam_esa.html

***

***

In [15]:
seed = 42
dataset_full_raw_dir = 'datasets/Sentinel2GlobalLULC_full_raw/Sentinel2LULC_JPEG/'

# Split the dataset folder

In [13]:
import splitfolders

In [14]:
# Split with a ratio.
# To only split into training and validation set, set a tuple to `ratio`, i.e, `(.8, .2)`.
splitfolders.ratio(data_dir, output="datasets/Sentinel2GlobalLULC_ratio",
                   seed=seed, ratio=(.7, .1, .2), group_prefix=None, move=False) # default values


Copying files: 0 files [00:00, ? files/s]
Copying files: 257 files [00:00, 2568.72 files/s]
Copying files: 614 files [00:00, 3157.54 files/s]
Copying files: 970 files [00:00, 3339.74 files/s]
Copying files: 1337 files [00:00, 3467.43 files/s]
Copying files: 1686 files [00:00, 3474.98 files/s]
Copying files: 2049 files [00:00, 3526.62 files/s]
Copying files: 2402 files [00:00, 3526.74 files/s]
Copying files: 2762 files [00:00, 3549.09 files/s]
Copying files: 3117 files [00:00, 3513.38 files/s]
Copying files: 3469 files [00:01, 3513.95 files/s]
Copying files: 3839 files [00:01, 3568.07 files/s]
Copying files: 4196 files [00:01, 3544.89 files/s]
Copying files: 4551 files [00:01, 3464.51 files/s]
Copying files: 4898 files [00:01, 2602.44 files/s]
Copying files: 5320 files [00:01, 2990.34 files/s]
Copying files: 5752 files [00:01, 3328.72 files/s]
Copying files: 6189 files [00:01, 3606.98 files/s]
Copying files: 6629 files [00:01, 3825.31 files/s]
Copying files: 7067 files [00:02, 3981.98 

In [6]:
# Split val/test with a fixed number of items, e.g. `(100, 100)`, for each set.
# To only split into training and validation set, use a single number to `fixed`, i.e., `10`.
# Set 3 values, e.g. `(300, 100, 100)`, to limit the number of training values.
# splitfolders.fixed(data_dir, output="datasets/Sentinel2GlobalLULC_balanced",
#                    seed=seed, fixed=(7250, 1160, 1740), oversample=True, group_prefix=None, move=False) # default values

In [10]:
#### Split val/test with a fixed number of items, e.g. `(100, 100)`, for each set.
# To only split into training and validation set, use a single number to `fixed`, i.e., `10`.
# Set 3 values, e.g. `(300, 100, 100)`, to limit the number of training values.
# splitfolders.fixed(data_dir, output="datasets/Sentinel2GlobalLULC_balanced",
#                    seed=seed, fixed=(400, 100), oversample=True, group_prefix=None, move=False) # default values


Copying files: 0 files [00:00, ? files/s]
Copying files: 1166 files [00:00, 11656.87 files/s]
Copying files: 2827 files [00:00, 14568.32 files/s]
Copying files: 4405 files [00:00, 15118.39 files/s]
Copying files: 5917 files [00:00, 11928.17 files/s]
Copying files: 7936 files [00:00, 14470.92 files/s]
Copying files: 9888 files [00:00, 16010.56 files/s]
Copying files: 11939 files [00:00, 17377.33 files/s]
Copying files: 14016 files [00:00, 18401.88 files/s]
Copying files: 15914 files [00:00, 18576.25 files/s]
Copying files: 18795 files [00:01, 16850.15 files/s]

Oversampling: 0 classes [00:00, ? classes/s]
Oversampling: 1 classes [00:00,  1.16 classes/s]
Oversampling: 2 classes [00:01,  1.29 classes/s]
Oversampling: 4 classes [00:02,  1.63 classes/s]


# Imports

In [4]:
import torch
import torchvision

import lightly
from lightly.models.modules.heads import SimSiamPredictionHead,SimSiamProjectionHead

import matplotlib.pyplot as plt
import numpy as np
import math

/home/sfandres/Documents/Git/lulc/lulc-venv/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


# Loading

In [5]:
data_dir = 'datasets/Sentinel2GlobalLULC-reduced/'
input_size = 222

In [6]:
# Torchvision custom transforms.
train_transform = torchvision.transforms.Compose([
    torchvision.transforms.Resize((input_size, input_size)),
    torchvision.transforms.RandomResizedCrop(size=input_size, scale=(0.2, 1.0)),
    torchvision.transforms.RandomHorizontalFlip(p=0.5),
    torchvision.transforms.RandomVerticalFlip(p=0.5),
    torchvision.transforms.GaussianBlur(21),
    torchvision.transforms.ToTensor(),
])

In [7]:
# Torchvision custom transforms.
test_transform = torchvision.transforms.Compose([
    torchvision.transforms.Resize((input_size, input_size)),
    torchvision.transforms.ToTensor(),
])

In [ ]:
train_data = datasets.ImageFolder(base_path + '/train/',
                                           transform=train_transform)
valid_data = datasets.ImageFolder(base_path + '/train/',
                                           transform=test_transform)